<a href="https://colab.research.google.com/github/PeterdeGuzman/PeterdeGuzmanMini9/blob/main/PeterdeGuzman_Mini9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDS 706 - Mini Project 9
#Cloud-Hosted Notebook Data Manipulation

## Peter de Guzman





### Project Explanation:

To explore the functionality of a Jupyter notebook hosted through Google Colab, I performed data cleaning operations on a dataset of U.S. counties from the U.S. Census Bureau to perform a crosswalk file that can be used for future data merging. This is a common task for analysts and researchers that use aggregated data for large or medium-size geographic areas in the United States.

### Project Requirements:

Requirements:
* Set up a cloud-hosted Jupyter Notebook (e.g., Google Colab)
* Perform data manipulation tasks on a sample dataset
* Test the operations in the Jupyter notebook as part of a CI/CD pipeline with GitHub Actions



First, we will load the data on U.S. counties from the U.S. Census Bureau. This dataset includes county and state names as well as two-digit state FIPS and three-digit county FIPS codes for each county in the U.S.

In [10]:
# Load data
import pandas as pd
import os

raw = pd.read_csv(
    "https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt",
    delimiter="|",
    header=0,
    dtype={"STATEFP": str, "COUNTYFP": str},
)

raw_coln = len(raw)

We will create multiple new fields in this dataset to support future merging operations. At each stage, we will test that the new created variables do not have missing rows and are successfully created.

The first variable we will create ia s 5-digit FIPS field by concatenating the two-digit State FIPS codes with the three-digit County FIPS codes.

In [11]:
raw["countyfips"] = raw["STATEFP"] + raw["COUNTYFP"]

We will add multiple assert statements to ensure that the new field was created successfully without adding missing data, creating invalid codes, or dropping any rows.

In [12]:
# assert there are no missing values
assert raw["countyfips"].isnull().sum() == 0
# assert that every 5 digit County FIPS field has 5 digits
assert all(
    raw["countyfips"].apply(lambda x: len(x) == 5 and x.isdigit())
), "Not all values are 5 digits"
# assert that the number of County Fips values has not changed
assert raw_coln == len(raw["countyfips"])

Some datasets do not include FIPS codes, but instead contain a field with county and state names such as "Durham County, NC". To join with these datasets, we will create a matching column for this crosswalk dataset. If there are differences in capitalization, you could use this column in the future by converting this column and its matching field into lowercase before merging.

In [13]:
# Create full County and State names field
raw["countynames_full"] = raw["COUNTYNAME"] + ", " + raw["STATE"]

We will again add multiple assert statements to ensure that the new field was created successfully without adding missing data, creating invalid codes, or dropping any rows.

In [14]:
# assert that there are no missing values
assert raw_coln == len(raw)

# assert that there are no missing values
assert raw["countynames_full"].isnull().sum() == 0

Unfortunately, many national datasets in the U.S. do not include data on Puerto Rico and other American territories. Therefore, we often need to filter out these territories and ensure that each row corresponds with a valid entry for the 50 states and Washington, D.C. The below code filters the dataset, checks that the appropriate rows were dropped, and then asserts that each row has a corresponding state value.

In [ ]:
# Filtering out U.S. territories that are not recognized states
# and are not needed for our analysis
# Puerto Rico, American Samoa, Guam,
# Northern Mariana Islands,
# United States Minor Outlying Islands
not_states = {"MP", "AS", "GU", "PR", "UM", "VI"}

# create regex pattern for checking
pattern = "|".join(not_states)
invalid_counties = raw[raw["STATE"].str.contains(pattern, na=False)].shape[0]

filtered_df = raw[~raw["STATE"].isin(not_states)].copy()

# Asserting that the rows were dropped
assert len(filtered_df) == raw_coln - invalid_counties

# assert that there every field has a state associated with it
state_abbr = {
    "AL",  # Alabama
    "AK",  # Alaska
    "AZ",  # Arizona
    "AR",  # Arkansas
    "CA",  # California
    "CO",  # Colorado
    "CT",  # Connecticut
    "DE",  # Delaware
    "DC",  # Washington, DC
    "FL",  # Florida
    "GA",  # Georgia
    "HI",  # Hawaii
    "ID",  # Idaho
    "IL",  # Illinois
    "IN",  # Indiana
    "IA",  # Iowa
    "KS",  # Kansas
    "KY",  # Kentucky
    "LA",  # Louisiana
    "ME",  # Maine
    "MD",  # Maryland
    "MA",  # Massachusetts
    "MI",  # Michigan
    "MN",  # Minnesota
    "MS",  # Mississippi
    "MO",  # Missouri
    "MT",  # Montana
    "NE",  # Nebraska
    "NV",  # Nevada
    "NH",  # New Hampshire
    "NJ",  # New Jersey
    "NM",  # New Mexico
    "NY",  # New York
    "NC",  # North Carolina
    "ND",  # North Dakota
    "OH",  # Ohio
    "OK",  # Oklahoma
    "OR",  # Oregon
    "PA",  # Pennsylvania
    "RI",  # Rhode Island
    "SC",  # South Carolina
    "SD",  # South Dakota
    "TN",  # Tennessee
    "TX",  # Texas
    "UT",  # Utah
    "VT",  # Vermont
    "VA",  # Virginia
    "WA",  # Washington
    "WV",  # West Virginia
    "WI",  # Wisconsin
    "WY",  # Wyoming
}

# Use a lambda function to check the last two letters
filtered_df.loc[:, "Valid_State"] = filtered_df["countynames_full"].apply(
    lambda county_state: county_state.split(", ")[-1] in state_abbr
)


# Assert that all entries are valid
assert all(
    filtered_df["Valid_State"]
), "Some entries do not have valid state abbreviations."

/var/folders/81/w_61xz297rv4ggdktb58tlxm0000gn/T/ipykernel_35150/3864709302.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, "Valid_State"] = filtered_df["countynames_full"].apply(


Finally, we will save the cleaned and filtered crosswalk dataset to a csv file for use in future data manipulation and merging operations. For this example, we will save it locally.

In [16]:
# Saving output to "00 - Resources"
filtered_df.to_csv(
    "fips_xwalk.csv",
    index=False,
)

Finally, we will assert that the file was successfully created.

In [17]:
file_path = "fips_xwalk.csv"
assert os.path.exists(file_path), "File does not exist."